기타 코드

In [ ]:
# 절대 건들지 말 것. 강제 종료 코드
# import os
# os.kill(os.getpid(), 9)

In [ ]:
# 구글 드라이브 데이터 불러오는 코드
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


- audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim

- 음성을 감정 분류 값으로 만드는 코드

- 해당 모델은 코드를 벡터화 해서 감정분류까지 해주는 코드
- k-wav2vec을 활용하는 코드는 별도의 코드로 할 예정
- 통합하니 너무 복잡해짐

In [ ]:
# 필요한 패키지 설치
!pip install regex==2024.11.6 \
numpy==1.26.4 \
setuptools==75.6.0 \
tokenizers==0.21.0 \
tqdm==4.67.1 \
transformers==4.47.0 \
librosa
!pip install jedi

In [ ]:
import os
import torch
import numpy as np
import soundfile as sf
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)
import torch.nn as nn
import librosa

In [ ]:
# Regression Head 정의
# - 마지막 레이어의 임베딩을 입력받아 arousal, dominance, valence를 예측하기 위한 레이어
class RegressionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        # 은닉 벡터 크기: config.hidden_size
        # 출력 레이어는 3차원 (arousal, dominance, valence)로 매핑
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)       # 드롭아웃: 과적합 방지
        x = self.dense(x)         # 선형 변환
        x = torch.tanh(x)         # 활성화 함수: tanh
        x = self.dropout(x)       # 다시 드롭아웃
        x = self.out_proj(x)      # 최종 출력층 -> 감정 값(3차원)
        return x

In [ ]:
# EmotionModel 정의
# - Wav2Vec2 모델을 상속받아 Emotion Regression Head를 붙인 모델
class EmotionModel(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        # Pretrained Wav2Vec2 모델 불러오기
        self.wav2vec2 = Wav2Vec2Model(config)
        # 감정 예측 레이어
        self.classifier = RegressionHead(config)
        # 가중치 초기화
        self.init_weights()

    def forward(self, input_values):
        # Wav2Vec2 모델에 입력: (batch, time)
        outputs = self.wav2vec2(input_values)
        # outputs[0]은 (batch, time, hidden_size) 형태의 last hidden states
        hidden_states = outputs[0]

        # 시간축에 대해 평균 풀링 -> (batch, hidden_size)
        hidden_states = torch.mean(hidden_states, dim=1)

        # 평균 풀링한 벡터를 Regression Head로 통과 -> (batch, 3)
        logits = self.classifier(hidden_states)

        # hidden_states: 임베딩, logits: 감정 예측값
        return hidden_states, logits

In [ ]:
# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모델 및 프로세서 로드
# 사용 모델: audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim
model_name = 'audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim'
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = EmotionModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

In [ ]:
# process_func 함수
# 주어진 waveform과 샘플링 레이트를 받아 모델 추론을 수행하는 함수.
# embeddings=True 시: hidden_states 반환
# embeddings=False 시: logits 반환
def process_func(waveform: np.ndarray, sampling_rate: int, embeddings: bool = False) -> np.ndarray:
    """
    오디오 신호에 대해 Wav2Vec2 기반 감정 추론을 수행하는 함수.
    Args:
        waveform (np.ndarray): 오디오 신호 (shape: (1, samples))
        sampling_rate (int): 오디오 샘플링 레이트 (예: 16kHz)
        embeddings (bool): True면 임베딩(hidden_states) 반환, False면 예측 감정(logits) 반환

    Returns:
        np.ndarray: 감정 예측값(배열) 또는 임베딩(배열)
    """

    # 1. Processor를 통해 입력값 전처리
    #    return_tensors="pt"로 파이토치 텐서 반환
    inputs = processor(waveform, sampling_rate=sampling_rate, return_tensors="pt")
    input_values = inputs["input_values"].to(device)

    # 2. 모델 추론 (no_grad로 백프로파게이션 비활성)
    with torch.no_grad():
        hidden_states, logits = model(input_values)

    # 3. embeddings 파라미터에 따라 반환값 선택
    output = hidden_states if embeddings else logits

    # CPU로 이동한 뒤 numpy로 변환
    return output.cpu().numpy()

In [ ]:
# resample_audio 함수
def resample_audio(filepath, target_sampling_rate=16000):
    try:
        waveform, original_sr = librosa.load(filepath, sr=None)
        print(f"Original sampling rate: {original_sr}")
        if original_sr != target_sampling_rate:
            waveform = librosa.resample(waveform, orig_sr=original_sr, target_sr=target_sampling_rate)
            print(f"Resampled to {target_sampling_rate} Hz")
        else:
            print("Sampling rate already matches the target rate.")
        return waveform, target_sampling_rate
    except Exception as e:
        print(f"Error resampling {filepath}: {e}")
        return None, None

In [ ]:
# analyze_all_files_with_resampling 함수
def analyze_all_files_with_resampling(directory: str):
    """
    지정된 디렉토리 내 모든 .wav 파일을 샘플링 레이트 변환 후 분석.
    """
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return

    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        if filename.lower().endswith(".wav"):
            try:
                print(f"Processing file: {filename}")

                # 오디오 샘플링 레이트 변환
                waveform, sr = resample_audio(filepath, target_sampling_rate=16000)
                if waveform is None:
                    continue

                # 모델에 입력 형식으로 변환
                waveform = waveform.reshape(1, -1).astype(np.float32)

                # 감정 예측 수행
                predictions = process_func(waveform, sr, embeddings=False)
                print(f"Predicted Arousal, Dominance, Valence for {filename}: {predictions[0]}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

In [ ]:
# analyze_three_folders 함수
def analyze_three_folders(original_dir: str, noise_dir: str, noise2_dir: str):
    """
    original, noise, noise2 폴더를 각각 분석하고, 대응하는 파일 간의 차이를 계산하여 결과를 저장.

    Args:
        original_dir (str): 원본 파일이 위치한 폴더 경로
        noise_dir (str): 첫 번째 노이즈 추가 파일이 위치한 폴더 경로
        noise2_dir (str): 두 번째 노이즈 추가 파일이 위치한 폴더 경로
    """
    # 디렉토리 존재 여부 확인
    if not os.path.exists(original_dir):
        print(f"Original directory '{original_dir}' does not exist.")
        return
    if not os.path.exists(noise_dir):
        print(f"Noise directory '{noise_dir}' does not exist.")
        return
    if not os.path.exists(noise2_dir):
        print(f"Noise2 directory '{noise2_dir}' does not exist.")
        return

    # 파일 목록 가져오기
    original_files = sorted([f for f in os.listdir(original_dir) if f.lower().endswith(".wav")])
    noise_files = sorted([f for f in os.listdir(noise_dir) if f.lower().endswith(".wav")])
    noise2_files = sorted([f for f in os.listdir(noise2_dir) if f.lower().endswith(".wav")])

    # 결과 저장 변수
    results = []

    for original_file in original_files:
        noise_file = original_file.replace(".wav", "n.wav")
        noise2_file = original_file.replace(".wav", "nn.wav")

        if noise_file in noise_files and noise2_file in noise2_files:
            try:
                print(f"Processing Original: {original_file}, Noise: {noise_file}, Noise2: {noise2_file}")
                original_path = os.path.join(original_dir, original_file)
                noise_path = os.path.join(noise_dir, noise_file)
                noise2_path = os.path.join(noise2_dir, noise2_file)

                # Original 파일 처리
                original_waveform, original_sr = resample_audio(original_path, target_sampling_rate=16000)
                if original_waveform is None:
                    continue
                original_waveform = original_waveform.reshape(1, -1).astype(np.float32)
                original_predictions = process_func(original_waveform, original_sr, embeddings=False)

                # Noise 파일 처리
                noise_waveform, noise_sr = resample_audio(noise_path, target_sampling_rate=16000)
                if noise_waveform is None:
                    continue
                noise_waveform = noise_waveform.reshape(1, -1).astype(np.float32)
                noise_predictions = process_func(noise_waveform, noise_sr, embeddings=False)

                # Noise2 파일 처리
                noise2_waveform, noise2_sr = resample_audio(noise2_path, target_sampling_rate=16000)
                if noise2_waveform is None:
                    continue
                noise2_waveform = noise2_waveform.reshape(1, -1).astype(np.float32)
                noise2_predictions = process_func(noise2_waveform, noise2_sr, embeddings=False)

                # 차이 계산
                diff_noise = np.abs(original_predictions[0] - noise_predictions[0])
                diff_noise2 = np.abs(original_predictions[0] - noise2_predictions[0])

                # 결과 저장
                results.append({
                    "original_file": original_file,
                    "noise_file": noise_file,
                    "noise2_file": noise2_file,
                    "original_predictions": original_predictions[0].tolist(),
                    "noise_predictions": noise_predictions[0].tolist(),
                    "noise2_predictions": noise2_predictions[0].tolist(),
                    "diff_noise": diff_noise.tolist(),
                    "diff_noise2": diff_noise2.tolist(),
                })

                # 출력
                print(f"Original Predictions: {original_predictions[0]}")
                print(f"Noise Predictions: {noise_predictions[0]}")
                print(f"Noise2 Predictions: {noise2_predictions[0]}")

            except Exception as e:
                print(f"Error processing {original_file}, {noise_file}, and {noise2_file}: {e}")
        else:
            print(f"Noise or Noise2 file for {original_file} not found in {noise_dir} or {noise2_dir}.")

    # 결과 반환
    return results

In [ ]:
import json

# JSON 저장 함수
def save_results_to_json(results, output_path):
    """
    분석 결과를 JSON 파일로 저장합니다.

    Args:
        results (list): 분석 결과 리스트
        output_path (str): 저장할 JSON 파일 경로
    """
    try:
        with open(output_path, 'w', encoding='utf-8') as json_file:
            json.dump(results, json_file, indent=4, ensure_ascii=False)  # ensure_ascii=False로 한글도 저장 가능
        print(f"Results successfully saved to {output_path}")
    except Exception as e:
        print(f"Error saving results to JSON: {e}")

- 메인 파트
- 메인파트에서 모두 실행하게 해줌

In [ ]:
# 메인 실행 부분
if __name__ == "__main__":
    base_dir = "/content/drive/My Drive/personal_study"
    original_directory = os.path.join(base_dir, "original")
    noise_directory = os.path.join(base_dir, "noise")
    noise2_directory = os.path.join(base_dir, "noise2")

    # 분석 함수 호출 및 결과 저장
    results = analyze_three_folders(original_directory, noise_directory, noise2_directory)

    # 결과 확인
    print("Analysis Complete. Results:")
    for result in results:
        print(result)

    # JSON 파일 경로 설정
    json_output_path = os.path.join(base_dir, "analysis_results-audeering.json")

    # JSON 파일로 저장
    save_results_to_json(results, json_output_path)

Processing Original: 1.wav, Noise: 1n.wav, Noise2: 1nn.wav
Original sampling rate: 16000
Sampling rate already matches the target rate.
Original sampling rate: 16000
Sampling rate already matches the target rate.
Original sampling rate: 16000
Sampling rate already matches the target rate.
Original Predictions: [0.6744778 0.6980703 0.5354375]
Noise Predictions: [0.6237564  0.64708865 0.5568416 ]
Noise2 Predictions: [0.6615995 0.6794806 0.5933319]
Processing Original: 10.wav, Noise: 10n.wav, Noise2: 10nn.wav
Original sampling rate: 16000
Sampling rate already matches the target rate.
Original sampling rate: 16000
Sampling rate already matches the target rate.
Original sampling rate: 16000
Sampling rate already matches the target rate.
Original Predictions: [0.5950483 0.6144976 0.4564102]
Noise Predictions: [0.5891726  0.54726934 0.49835497]
Noise2 Predictions: [0.637548   0.6308239  0.42813274]
Processing Original: 2.wav, Noise: 2n.wav, Noise2: 2nn.wav
Original sampling rate: 16000
Sampl

수치들을 3차원 값으로 보여주는 코드

In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objects as go

def visualize_emotion_results_with_ticks(results):
    """
    analyze_three_folders 결과 데이터를 기반으로 Original, Noise, Noise2 데이터를 시각화.
    축 눈금을 0.2 단위로 설정.

    Args:
        results (list of dict): analyze_three_folders 함수의 결과로 생성된 리스트.
    """
    # 데이터 분리
    original_data = [result["original_predictions"] for result in results]
    noise_data = [result["noise_predictions"] for result in results]
    noise2_data = [result["noise2_predictions"] for result in results]
    filenames = [result["original_file"] for result in results]

    # Original 데이터 분리
    arousal_org = [data[0] for data in original_data]
    dominance_org = [data[1] for data in original_data]
    valence_org = [data[2] for data in original_data]

    # Noise 데이터 분리
    arousal_noise = [data[0] for data in noise_data]
    dominance_noise = [data[1] for data in noise_data]
    valence_noise = [data[2] for data in noise_data]

    # Noise2 데이터 분리
    arousal_noise2 = [data[0] for data in noise2_data]
    dominance_noise2 = [data[1] for data in noise2_data]
    valence_noise2 = [data[2] for data in noise2_data]

    # 3D 그래프 생성
    fig = go.Figure()

    # Original 데이터 추가 (빨간색 계열)
    fig.add_trace(go.Scatter3d(
        x=arousal_org,
        y=dominance_org,
        z=valence_org,
        mode='markers+text',
        marker=dict(
            size=5,
            color='red',
            opacity=0.8
        ),
        text=filenames,  # 파일 이름 레이블
        textposition="top center",
        name="Original"
    ))

    # Noise 데이터 추가 (파란색 계열)
    fig.add_trace(go.Scatter3d(
        x=arousal_noise,
        y=dominance_noise,
        z=valence_noise,
        mode='markers+text',
        marker=dict(
            size=5,
            color='blue',
            opacity=0.8
        ),
        text=filenames,  # 파일 이름 레이블
        textposition="top center",
        name="Noise"
    ))

    # Noise2 데이터 추가 (초록색 계열)
    fig.add_trace(go.Scatter3d(
        x=arousal_noise2,
        y=dominance_noise2,
        z=valence_noise2,
        mode='markers+text',
        marker=dict(
            size=5,
            color='green',
            opacity=0.8
        ),
        text=filenames,  # 파일 이름 레이블
        textposition="top center",
        name="Noise2"
    ))

    # 레이아웃 설정 (축 눈금을 0.2 단위로 설정)
    fig.update_layout(
        title="3D Emotion Graph with 0.2 Tick Intervals",
        scene=dict(
            xaxis=dict(
                title='Arousal',
                range=[0, 1],
                tickmode='linear',  # 선형 눈금
                tick0=0,            # 시작값
                dtick=0.2           # 눈금 간격
            ),
            yaxis=dict(
                title='Dominance',
                range=[0, 1],
                tickmode='linear',
                tick0=0,
                dtick=0.2
            ),
            zaxis=dict(
                title='Valence',
                range=[0, 1],
                tickmode='linear',
                tick0=0,
                dtick=0.2
            ),
        ),
        margin=dict(l=0, r=0, b=0, t=40)  # 그래프 여백
    )

    # 그래프 표시
    fig.show()

# 분석 결과로 시각화 호출
visualize_emotion_results_with_ticks(results)
